In [ ]:
#Code to Calculate Mie Theory for Core-Shell spheres using theory from 
#"Improved recursive algorithm for light scattering by a multilayered sphere" from Applied Optics Yang 2003

import time
import pandas as pd
import scipy.constants as sc
import scipy.special
import scipy
import datetime
import numpy as np
import math

#File containing refractive indices of material
files = []
fileCount = 0
print("The following text files have been detected:\n")
for file in os.listdir(os.getcwd()):
     # check the files which are end with specific extension
    if file.endswith(".txt"):
        # print path name of selected files
        print("[",fileCount,"] ",file)
        files.append(file)
        fileCount += 1
filename = int(input("\nPlease select file number:"))
inputdata = os.getcwd() + "/" + files[filename] #path of the file
print(inputdata)
#Note: all refractive indexes must be in the same text file

data1 = pd.read_csv(inputdata, sep='\t', header=None, skiprows=1).values 
lamda = data1[:,0] #wavelengths in nm
lamda[0]=round(lamda[0]) #For purpose of Interpolation() fxn

n_m = 1.33 #real part of the refractive index of medium
k_m = 0 #imaginary part of refractive index of medium
N_m = n_m+(k_m*1j)
m_m = 1 #relative refractive index of medium

L = int(input("Layers:\t")) #Ask user for the amount of layers (Requirment: Interger, Greater that 1)
radii, mi, columns = [], [], []
for i in range(1, L+1):
    if(i==L):
        temp = float(input("Outer radius of shell "+str(i)+" (Outermost Shell):\t"))
    elif(i==1):
        temp = float(input("Outer radius of shell "+str(i)+" (Innermost Core):\t"))
    else:
        temp = float(input("Outer radius of shell "+str(i)+":\t"))
    radii.append(temp) 
for i in range(1, L+1):
    temp1 = int(input("Column contaning "+ '\033[1m' + 'real' + '\033[0m'+" refrac index of shell "+str(i)+":\t"))
    temp2 = int(input("Column contaning "+ '\033[1m' + 'imaginary' + '\033[0m'+" refrac index of shell "+str(i)+":\t"))
    temp = (data1[:,temp1] + (data1[:,temp2]*1j))/N_m
    columns.append(temp1)
    columns.append(temp2)
    mi.append(temp)                                

#This funciton returns the linear interpolation of the refractive index between two wavelengths
def interpolation(x, y):
    for i in range(0,len(lamda)):
        if(lamda[i] < x and lamda[i+1] >= x):
            j=i
            break
    return y[j]+(((y[j+1]-y[j])/(lamda[j+1]-lamda[j]))*(x-lamda[j]))

interpol = int(input("How would you like to have your wavelength intervals from your refractive index (dielectric fxn) file?\n\t[1] Use refractive index file as is.\n\t[2] Interpolate dielectric values between wavelengths.\n"))
if(interpol == 2):
    interval = str(input("What would you like the interval between wavelengths to be?\n\tNote: Wavelenghts can only be interpolated up to the highest wavelength in your dielectric file.\n\tDefault: 1nm intervals\n"))
    if(interval==""):
        interval=1
    else:
        interval=float(interval)
    mi_new=[]
    curr = lamda[0]+interval
    lamda_new=np.array(lamda[0])
    for g in np.arange(0,int(len(columns)/2)):
        ml = [mi[g][0]]
        for i in np.arange(lamda[0],lamda[len(lamda)-1], interval):
            if(curr>lamda[len(lamda)-1]):
                break
            temp1 = interpolation(curr,mi[g].real)
            temp2 = interpolation(curr,mi[g].imag)
            ml.append((temp1+(temp2*1j)))
            curr = curr+interval
        ml.append(mi[g][len(mi[g])-1])
        mi_new.append(np.array(ml))
        lamda_new = np.arange(lamda[0],lamda[len(lamda)-1], interval)
        np.insert(lamda_new, 0, lamda[0])
        lamda_new = np.append(lamda_new,lamda[len(lamda)-1])
        curr = lamda[0]+interval
    mi = mi_new
    lamda = lamda_new
    
m = np.array(mi[0]).reshape(len(mi[0]),1)
for i in range(1, L):
    m = np.hstack((m,np.array(mi[i]).reshape(len(mi[i]),1)))   
m = np.hstack((m,np.ones((len(m), 1)))) #last column is relative refractive index of medium

def rA(l, i): #Radii Adjustment
    # l = index of desired refractive index (innermost = 1)
    # i = index of radius (innermost = 1)
    # The idea of this function is to produce m_l*r_i for any l and i
    return (m[:,l-1]*2*sc.pi*radii[i-1]*N_m)/lamda

#The next two variables are chosen from Eq 30 from Yang 2003
largest=0
for j in range(1,L+1):
    if(max(rA(j,j).real) > largest):
        largest = max(rA(j,j).real)
    if(max(rA(j+1,j).real) > largest):
        largest = max(rA(j+1,j).real)
lastTerm = math.ceil(abs(largest+4.05*(largest**(1/3))+2))

def phi(n,z): 
    return z*scipy.special.spherical_jn(n,z)
    #Spherical Bessel Function where n is the order and z is the parameter

def Dphi(n,z): 
    return (z*scipy.special.spherical_jn(n,z, True))+(scipy.special.spherical_jn(n,z))
    #Derivative of phi using product rule

def spherical_hn1(n,z):
    return scipy.special.spherical_jn(n,z)+(1j*scipy.special.spherical_yn(n,z))
    #Spherical Hankel function (not in a python library, had to find alternate definition https://en.wikipedia.org/wiki/Bessel_function#Hankel_functions:_H(1)α,_H(2)α)

def SphericalHankelH2(n,z):
    return scipy.special.spherical_jn(n,z)-(1j*scipy.special.spherical_yn(n,z))
    #from https://en.wikipedia.org/wiki/Bessel_function#Hankel_functions:_H(1)α,_H(2)α

def zeta(n,z): 
    return z*(spherical_hn1(n,z))

def Dzeta(n,z): 
    return (scipy.special.spherical_jn(n,z)+(1j*scipy.special.spherical_yn(n,z)))+(z*(-spherical_hn1(n+1,z)+(n/z)*spherical_hn1(n,z)))

def D1(n,z): 
    return Dphi(n,z)/phi(n,z)

def D3(n,z): 
    return Dzeta(n,z)/zeta(n,z)

def RR(n,z): 
    return phi(n,z)/zeta(n,z)

def Hal(n, l):
    return ((RR(n,rA(l,l))*D1(n,rA(l,l)))-(Al(n,l)*D3(n,rA(l,l))))/(RR(n,rA(l,l))-Al(n,l))
    
def Al(n, l):
    if(l==2):
        return (RR(n,rA(2,1)))*(((m[:,1]*D1(n,rA(1,1)))-(m[:,0]*D1(n,rA(2,1))))/((m[:,1]*D1(n,rA(1,1)))-(m[:,0]*D3(n,rA(2,1)))))
    else:
        return (RR(n,rA(l,l-1)))*(((m[:,l-1]*Hal(n,l-1))-(m[:,l-2]*D1(n,rA(l,l-1))))/((m[:,l-1]*Hal(n,l-1))-(m[:,l-2]*D3(n,rA(l,l-1)))))

def Hbl(n, l):
    return ((RR(n,rA(l,l))*D1(n,rA(l,l)))-(Bl(n,l)*D3(n,rA(l,l))))/(RR(n,rA(l,l))-Bl(n,l))

def Bl(n, l):
    if(l==2):
        return (RR(n,rA(2,1)))*(((m[:,0]*D1(n,rA(1,1)))-(m[:,1]*D1(n,rA(2,1))))/((m[:,0]*D1(n,rA(1,1)))-(m[:,1]*D3(n,rA(2,1)))))
    else:
        return (RR(n,rA(l,l-1)))*(((m[:,l-2]*Hbl(n,l-1))-(m[:,l-1]*D1(n,rA(l,l-1))))/((m[:,l-2]*Hbl(n,l-1))-(m[:,l-1]*D3(n,rA(l,l-1)))))

kappa = (2*sc.pi*N_m)/lamda

def cext(L): 
    sumd=0
    for n in np.arange(1, lastTerm+1):
        sumd=sumd+((2*n)+1)*(Al(n,L+1).real+Bl(n,L+1).real)
    return ((2*sc.pi*sumd)/kappa**2).real

def csca(L):
    sumd=0
    for n in np.arange(1, lastTerm+1):
        sumd=sumd+((2*n)+1)*((np.abs(Al(n,L+1))**2)+(np.abs(Bl(n,L+1))**2))
    return ((2*sc.pi*sumd)/kappa**2).real

def cabs(L): 
    return cext(L)-csca(L)

def Qext(L):
    i = L-1
    #incase any of the outer layers are the same as the medium
    while i>(-1):
        if (m[:,i]!=m_m).all():
            solid = i #set the solid outer layer index (core = 1)
            break
        i-=1
    if(solid==0):
        return 0
    else:
        return (cext(L))/(sc.pi*(radii[solid]**2))
    
def Qsca(L):
    i = L-1
    #incase any of the outer layers are the same as the medium
    while i>(-1):
        if (m[:,i]!=m_m).all():
            solid = i #set the solid outer layer index (core = 1)
            break
        i-=1
    if(solid==0):
        return 0
    else:
        return (csca(L))/(sc.pi*(radii[solid]**2))

def Qabs(L): 
    return Qext(L)-Qsca(L)

def Qnf(l):
    sumd = 0
    for n in np.arange(1, lastTerm+1):
        sumd = sumd + (((np.abs(Al(n,l)))**2)*(((n+1)*(np.abs(SphericalHankelH2(n-1,rA(l,l-1))))**2)+(n*(np.abs(SphericalHankelH2(n+1,rA(l,l-1))))**2)))+(((2*n)+1)*((np.abs(Bl(n,l)))**2)*(np.abs(SphericalHankelH2(n,rA(l,l-1))))**2)
    return 0.5*sumd

date = datetime.datetime.now()
date = date.strftime("%x")
date = date.replace("/","")
radList=str(radii[0])
for i in range(1, L):
    radList = "_".join([radList,str(radii[i])])

outputFile = 'mie_EXTABSNF_'+date+"_"+radList+'.txt'

file = open(outputFile, "w+")

qnf_header="Qnf_1"
if(L>1):
    for i in range(2,L+1):
        qnf_header = "\t".join([qnf_header,"Qnf_"+str(i)])
header = "\t".join(["Lambda","Qext","Qsca","Qabs","Cext","Csca","Cabs",qnf_header])

file.write(header)
file.write("\n")

start_time = time.time()
    
list1 = lamda.real
list2 = Qext(L).real
list3 = Qsca(L).real
list4 = Qabs(L).real
list5 = cext(L).real
list6 = csca(L).real
list7 = cabs(L).real
Qnfl = np.array(Qnf(2)).reshape(len(Qnf(2)),1)
if(L>1):
    for i in range(3,L+2):
        Qnfl = np.hstack((Qnfl, np.array(Qnf(i)).reshape(len(Qnf(i)),1)))
    
for i in range(0,len(lamda)):
    QnfText = str(Qnfl[i][0])
    for j in range(1, Qnfl.shape[1]):
        QnfText = "\t".join([QnfText,str(Qnfl[i][j])])
    text = "\t".join([str(list1[i]),
                     str(list2[i]),
                     str(list3[i]),
                     str(list4[i]),
                     str(list5[i]),
                     str(list6[i]),
                     str(list7[i]),
                     str(QnfText)])
    file.write(text+"\n")
file.close()

print("\n\nMy program took", time.time() - start_time,"seconds to run")